## IBM Applied Data Science Capstone Project by Coursera
### Week 5 Final Report
### Opening a New Cafe Coffee Shop in Bangalore,India.

* Build a dataframe of neighborhoods in Bangalore, India by web scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall

### 1. Import libraries

In [120]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [121]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore").text

In [122]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [123]:

# create a list to store neighborhood data
neighborhoodList = []

In [124]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [129]:
# create a new DataFrame from the list
b_df = pd.DataFrame({"Neighborhood": neighborhoodList})

b_df.head()

,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"


#### Dropping those rows which just contain a heading as row 

In [130]:
b_df = b_df.drop([0,1,2] , axis = 0)
b_df.head()

,Neighborhood
3,Adugodi
4,"Agara, Bangalore"
5,Ananthnagar
6,Anjanapura
7,Arekere


In [131]:
# print the number of rows of the dataframe
b_df.shape

(128, 1)

### 3. Get the geographical coordinates

In [132]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore , India '.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [133]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in b_df["Neighborhood"].tolist() ]

In [134]:
coords

[[12.94404000000003, 77.60795000000007],
 [12.842700000000036, 77.48882000000003],
 [12.954080000000033, 77.54129000000006],
 [12.858100000000036, 77.55906000000004],
 [12.885640000000024, 77.59669000000008],
 [12.963430000000074, 77.61294000000004],
 [13.027520000000038, 77.65044000000006],
 [13.077252562687073, 77.65756129813772],
 [13.044700000000034, 77.55000000000007],
 [12.922280000000057, 77.56986000000006],
 [13.028473466463632, 77.63189195846024],
 [12.939000000000021, 77.57135000000005],
 [12.992260000000044, 77.53441000000004],
 [12.927350000000047, 77.67184000000003],
 [13.082990000000052, 77.54779000000008],
 [12.900070000000028, 77.60431000000005],
 [12.90307000000007, 77.62441000000007],
 [12.817540000000065, 77.67879000000005],
 [12.993280000000027, 77.66127000000006],
 [12.91489000000007, 77.61004000000008],
 [12.951800000000048, 77.54438000000005],
 [12.913190000000043, 77.71554000000003],
 [12.959686963552544, 77.57131311818829],
 [12.793990000000065, 77.700180000000

In [135]:

# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [136]:
# merge the coordinates into the original dataframe
b_df['Latitude'] = df_coords['Latitude']
b_df['Longitude'] = df_coords['Longitude']

In [137]:
# check the neighborhoods and the coordinates
print(b_df.shape)
b_df

(128, 3)


,Neighborhood,Latitude,Longitude
3,Adugodi,12.858100,77.559060
4,"Agara, Bangalore",12.885640,77.596690
5,Ananthnagar,12.963430,77.612940
6,Anjanapura,13.027520,77.650440
7,Arekere,13.077253,77.657561
8,Austin Town,13.044700,77.550000
9,Babusapalya,12.922280,77.569860
10,"Bagalur, Bangalore Urban",13.028473,77.631892
11,Bahubalinagar,12.939000,77.571350
12,Banashankari,12.992260,77.534410


#### Drop those which couldnt be identified as co ordinates 

In [138]:
b_df = b_df.dropna()
b_df

,Neighborhood,Latitude,Longitude
3,Adugodi,12.858100,77.559060
4,"Agara, Bangalore",12.885640,77.596690
5,Ananthnagar,12.963430,77.612940
6,Anjanapura,13.027520,77.650440
7,Arekere,13.077253,77.657561
8,Austin Town,13.044700,77.550000
9,Babusapalya,12.922280,77.569860
10,"Bagalur, Bangalore Urban",13.028473,77.631892
11,Bahubalinagar,12.939000,77.571350
12,Banashankari,12.992260,77.534410


In [139]:
# save the DataFrame as CSV file
b_df.to_csv("b_df.csv", index=False)

### 4. Create a map of Bangalore with neighborhoods superimposed on top

In [140]:
# get the coordinates of Bangalore
address = 'Bangalore , India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore , India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore , India 12.9791198, 77.5912997.


In [141]:
# create map of Bangalore using latitude and longitude values
map_b = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(b_df['Latitude'], b_df['Longitude'], b_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_b)  
    
map_b

In [142]:
# save the map as HTML file
map_b.save('map_bangalore.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [143]:

# define Foursquare Credentials and Version
CLIENT_ID = 'JWHCIWFXNDRV1S5MIIXRB50UIYUIDU1UD42XE55GRCNBSFLZ' # your Foursquare ID
CLIENT_SECRET = 'TCNLLPCMVDMJXMMJRMPQHL3XO41OIFJ2ZREHG3G2D5D5UTWH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JWHCIWFXNDRV1S5MIIXRB50UIYUIDU1UD42XE55GRCNBSFLZ
CLIENT_SECRET:TCNLLPCMVDMJXMMJRMPQHL3XO41OIFJ2ZREHG3G2D5D5UTWH


#### Now, let's get the top 100 venues that are within a radius of 2000 meters

In [144]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(b_df['Latitude'], b_df['Longitude'], b_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [145]:

# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6679, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.8581,77.55906,capital club resorts,12.855979,77.555653,Pool
1,Adugodi,12.8581,77.55906,Chenrayappa Quarry,12.856130,77.565291,Campground
2,Adugodi,12.8581,77.55906,Vikash Kumar Photography,12.867472,77.566202,Photography Lab
3,Adugodi,12.8581,77.55906,ABCP Badminton,12.863512,77.571274,Badminton Court
4,Adugodi,12.8581,77.55906,Baristra,12.848037,77.568779,Coffee Shop


#### Let's check how many venues were returned for each neighorhood

In [146]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,5,5,5,5,5,5
"Agara, Bangalore",83,83,83,83,83,83
Ananthnagar,100,100,100,100,100,100
Anjanapura,52,52,52,52,52,52
Arekere,13,13,13,13,13,13
Austin Town,19,19,19,19,19,19
BTM Layout,80,80,80,80,80,80
Babusapalya,100,100,100,100,100,100
"Bagalur, Bangalore Urban",55,55,55,55,55,55


In [147]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 233 uniques categories.


In [148]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pool', 'Campground', 'Photography Lab', 'Badminton Court',
       'Coffee Shop', 'Sporting Goods Shop', 'Ice Cream Shop',
       'Multiplex', 'BBQ Joint', 'South Indian Restaurant', 'Lounge',
       'Indian Restaurant', 'Beer Garden', 'Shopping Mall',
       'Bowling Alley', 'Restaurant', 'Chinese Restaurant', 'Pizza Place',
       'Café', 'Sandwich Place', 'General Entertainment',
       'Department Store', 'Office', 'Liquor Store',
       'Fast Food Restaurant', 'Eastern European Restaurant',
       'American Restaurant', 'Middle Eastern Restaurant',
       'Rajasthani Restaurant', 'Burger Joint', 'Supermarket',
       'Italian Restaurant', 'Dumpling Restaurant', 'Dive Bar',
       'Electronics Store', 'Andhra Restaurant', 'Gym', 'Clothing Store',
       'Movie Theater', 'Tea Room', 'Cosmetics Shop', 'Bookstore',
       'Vegetarian / Vegan Restaurant', 'Breakfast Spot', 'Park',
       'Food Court', 'Kebab Restaurant', 'Bakery', 'Gift Shop',
       'Food Truck'], dtype=object)

In [149]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [150]:
# one hot encoding
b_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
b_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [b_onehot.columns[-1]] + list(b_onehot.columns[:-1])
b_onehot = b_onehot[fixed_columns]

print(b_onehot.shape)
b_onehot.head()

(6679, 234)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Apres Ski Bar,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Photography Lab,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [151]:
b_grouped = b_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(b_grouped.shape)
b_grouped

(125, 234)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Apres Ski Bar,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,Hockey Arena,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Parsi Restaurant,Performing Arts Venue,Pharmacy,Photography Lab,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Print Shop,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Toll Booth,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [152]:
len(b_grouped[b_grouped["Coffee Shop"] > 0])

95

#### Create new Dataframe for Coffee shops only 

In [153]:
b_coffee_shop = b_grouped[["Neighborhoods","Coffee Shop"]]

In [154]:
b_coffee_shop.head()

,Neighborhoods,Coffee Shop
0,Adugodi,0.200000
1,"Agara, Bangalore",0.012048
2,Ananthnagar,0.020000
3,Anjanapura,0.038462
4,Arekere,0.076923


### 7. Cluster Neighborhoods

In [155]:
# set number of clusters
kclusters = 4

b_clustering = b_coffee_shop.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(b_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 2, 2, 0, 1, 2, 0, 0, 2, 0], dtype=int32)

In [156]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
b_merged = b_coffee_shop.copy()

# add clustering labels
b_merged["Cluster Labels"] = kmeans.labels_

In [157]:

b_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
b_merged.head()

,Neighborhood,Coffee Shop,Cluster Labels
0,Adugodi,0.200000,3
1,"Agara, Bangalore",0.012048,2
2,Ananthnagar,0.020000,2
3,Anjanapura,0.038462,0
4,Arekere,0.076923,1


In [158]:
# merge bangalore with bangalore_data to add latitude/longitude for each neighborhood
b_merged = b_merged.join(b_df.set_index("Neighborhood"), on="Neighborhood")

print(b_merged.shape)
b_merged.head() # check the last columns!

(125, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Adugodi,0.200000,3,12.858100,77.559060
1,"Agara, Bangalore",0.012048,2,12.885640,77.596690
2,Ananthnagar,0.020000,2,12.963430,77.612940
3,Anjanapura,0.038462,0,13.027520,77.650440
4,Arekere,0.076923,1,13.077253,77.657561


In [159]:
# sort the results by Cluster Labels
print(b_merged.shape)
b_merged.sort_values(["Cluster Labels"], inplace=True)
b_merged

(125, 5)


,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
124,Wilson Garden,0.066667,0,13.029540,77.540220
35,"Fraser Town, Bangalore",0.038961,0,13.014450,77.588090
109,Suddaguntepalya,0.030000,0,12.922350,77.610280
38,"Ganganagar, Bangalore",0.040000,0,12.943710,77.561190
83,Murugeshpalya,0.036145,0,13.014800,77.538900
43,"Hanumanthanagar, Bengaluru",0.046512,0,13.049690,77.589510
49,Immadihalli,0.050000,0,12.908310,77.590170
100,Sadashivanagar,0.060000,0,12.993570,77.579890
52,J. P. Nagar,0.030000,0,12.928700,77.582790
98,Ramanjaneyanagar,0.025641,0,13.062720,77.585480


#### Visualize resulting Clusters

In [160]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(b_merged['Latitude'], b_merged['Longitude'], b_merged['Neighborhood'], b_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [161]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters 

#### Cluster 0 

In [162]:
b_merged.loc[b_merged['Cluster Labels'] == 0]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
124,Wilson Garden,0.066667,0,13.029540,77.540220
35,"Fraser Town, Bangalore",0.038961,0,13.014450,77.588090
109,Suddaguntepalya,0.030000,0,12.922350,77.610280
38,"Ganganagar, Bangalore",0.040000,0,12.943710,77.561190
83,Murugeshpalya,0.036145,0,13.014800,77.538900
43,"Hanumanthanagar, Bengaluru",0.046512,0,13.049690,77.589510
49,Immadihalli,0.050000,0,12.908310,77.590170
100,Sadashivanagar,0.060000,0,12.993570,77.579890
52,J. P. Nagar,0.030000,0,12.928700,77.582790
98,Ramanjaneyanagar,0.025641,0,13.062720,77.585480


#### Cluster 1 

In [163]:
b_merged.loc[b_merged['Cluster Labels'] == 1]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
74,Lingarajapuram,0.070000,1,13.006300,77.568289
59,Kalasipalyam,0.076923,1,12.968190,77.521140
12,Basavanagudi,0.125000,1,13.082990,77.547790
4,Arekere,0.076923,1,13.077253,77.657561
86,Nandini Layout,0.073684,1,12.998975,77.574984
15,Bharathnagar,0.142857,1,12.817540,77.678790
16,Bilekahalli,0.071429,1,12.993280,77.661270
92,R. T. Nagar,0.100000,1,13.061250,77.551110
31,Dollars Colony,0.106383,1,12.845470,77.664300
55,"Jayanagar, Bangalore",0.095238,1,12.984820,77.675030


#### Cluster 2 

In [164]:
b_merged.loc[b_merged['Cluster Labels'] == 2]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
96,Ramagondanahalli,0.020000,2,12.962710,77.601190
85,Nagarbhavi,0.016949,2,12.915470,77.553110
113,Thindlu,0.000000,2,12.897600,77.528220
122,Vyalikaval,0.019608,2,12.900420,77.568630
121,Vimanapura,0.014493,2,12.949620,77.596750
108,"Statue of Queen Victoria, Bangalore",0.019231,2,13.007705,77.612809
105,"Shivajinagar, Bangalore",0.010000,2,12.956398,77.638392
114,Thyagarajanagar,0.000000,2,12.943480,77.747030
94,"Rajarajeshwari Nagar, Bangalore",0.000000,2,13.023780,77.677870
104,Shikaripalya,0.010000,2,12.966870,77.587340


#### Cluster 3 

In [165]:
b_merged.loc[b_merged['Cluster Labels'] == 3]

,Neighborhood,Coffee Shop,Cluster Labels,Latitude,Longitude
44,Hebbagodi,0.20,3,13.03868,77.66187
57,Jigani,0.20,3,13.01061,77.49067
20,Byatarayanapura,0.25,3,12.79399,77.70018
0,Adugodi,0.20,3,12.85810,77.55906


#### Observations

Most of the Coffee Shops are concentrated in the central area of Bangalore city, with the highest number in cluster 2 and moderate number in cluster 1 and 3 . This represents a great opportunity and high potential areas to open new Coffee Shops as there is very little competition from existing shops. Meanwhile, Cafe Coffee Shops in cluster 2 are likely suffering from intense competition due to oversupply and high concentration of shops. From another perspective, this also shows that the oversupply of cafe shops mostly happened in the central area of the city, with the suburb area still have very few Cafe Coffee Shops. Therefore, this project recommends property developers to capitalize on these findings to open new CCD coffee shop in neighborhoods in cluster 3 with little competition. Property developers with unique selling propositions to stand out from the competition can also open new shops in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 0 and 2 which already have high concentration of shops and suffering from intense competition.